# 🏀 Joshua Dworkin — Career Tracker

A dedicated analysis tracking Joshua's progression through EDJBA seasons, with percentile rankings against his age-group peers.

---

In [ ]:
%matplotlib inline
import sqlite3, re, pandas as pd, numpy as np
import matplotlib.pyplot as plt, seaborn as sns
import warnings; warnings.filterwarnings('ignore')

sns.set_theme(style="whitegrid"); plt.rcParams['figure.dpi'] = 120

SEASON_ORDER = {
    "Summer 2020/21": 0, "Winter 2021": 1, "Summer 2021/22": 2, "Winter 2022": 3,
    "Summer 2022/23": 4, "Winter 2023": 5, "Summer 2023/24": 6, "Winter 2024": 7,
    "Summer 2024/25": 8, "Winter 2025": 9, "Summer 2025/26": 10,
}

conn = sqlite3.connect("../data/playhq.db")
all_stats = pd.read_sql("""
    SELECT p.id as pid, p.first_name || ' ' || p.last_name as name,
        s.name as season, g.name as grade,
        ps.games_played as gp, ps.total_points as pts,
        ps.one_point as ft, ps.two_point as fg2, ps.three_point as fg3,
        ps.total_fouls as fouls, ps.team_name as team
    FROM player_stats ps
    JOIN players p ON p.id = ps.player_id
    JOIN grades g ON g.id = ps.grade_id
    JOIN seasons s ON s.id = g.season_id
    WHERE ps.games_played > 0
""", conn)
conn.close()

all_stats["ppg"] = all_stats["pts"] / all_stats["gp"]
all_stats["fpg"] = all_stats["fouls"] / all_stats["gp"]
all_stats["age_group"] = all_stats["grade"].str.extract(r'(U\d+)')
all_stats["season_order"] = all_stats["season"].map(SEASON_ORDER)

josh = all_stats[all_stats["name"] == "Joshua Dworkin"].copy()
print(f"Joshua's stat lines: {len(josh)}")

## Season-by-Season Stats

In [ ]:
js = josh.groupby(["season", "age_group"]).agg(
    gp=("gp", "sum"), pts=("pts", "sum"), ft=("ft", "sum"),
    fg2=("fg2", "sum"), fg3=("fg3", "sum"), fouls=("fouls", "sum"),
    season_order=("season_order", "first"), team=("team", "first")
).reset_index().sort_values("season_order")

js["ppg"] = js["pts"] / js["gp"]
js["fpg"] = js["fouls"] / js["gp"]

display(js[["season", "age_group", "team", "gp", "pts", "ppg", "fpg", "ft", "fg2", "fg3"]].round(1))

## PPG Progression

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

ax = axes[0]
ax.plot(js["season"], js["ppg"], "o-", color="#1976D2", lw=2.5, markersize=8)
ax.fill_between(js["season"], js["ppg"], alpha=0.15, color="#1976D2")
ax.set_title("Joshua's PPG Over Time", fontweight="bold")
ax.set_ylabel("Points Per Game"); ax.tick_params(axis='x', rotation=45)
for _, r in js.iterrows():
    ax.annotate(f'{r["ppg"]:.1f}', (r["season"], r["ppg"]), textcoords="offset points",
                xytext=(0, 10), ha='center', fontsize=9)

ax = axes[1]
width = 0.25
x = np.arange(len(js))
ax.bar(x - width, js["ft"], width, label="FT", color="#4CAF50")
ax.bar(x, js["fg2"], width, label="2PT Makes", color="#1976D2")
ax.bar(x + width, js["fg3"], width, label="3PT Makes", color="#D32F2F")
ax.set_xticks(x); ax.set_xticklabels(js["season"], rotation=45, ha="right")
ax.set_title("Shooting Breakdown by Season", fontweight="bold")
ax.set_ylabel("Makes"); ax.legend()

plt.tight_layout()
plt.savefig("../assets/joshua_career.png", dpi=150, bbox_inches="tight")
plt.show()

## Percentile Rankings vs Age Group Peers

In [ ]:
# Calculate Joshua's percentile in each age group
agg = all_stats.groupby(["pid", "name", "age_group"]).agg(
    gp=("gp", "sum"), pts=("pts", "sum")
).reset_index()
agg["ppg"] = agg["pts"] / agg["gp"]
agg = agg[agg["gp"] >= 3]

josh_pct = []
for _, jr in agg[agg["name"] == "Joshua Dworkin"].iterrows():
    peers = agg[agg["age_group"] == jr["age_group"]]
    pct = (peers["ppg"] < jr["ppg"]).mean() * 100
    josh_pct.append({"Age Group": jr["age_group"], "PPG": round(jr["ppg"], 1),
                     "Percentile": round(pct, 0), "Peers": len(peers)})

pct_df = pd.DataFrame(josh_pct)
display(pct_df)

if len(pct_df) > 0:
    fig, ax = plt.subplots(figsize=(8, 4))
    bars = ax.bar(pct_df["Age Group"], pct_df["Percentile"], color="#1976D2", edgecolor="white")
    ax.axhline(50, color="#9E9E9E", ls="--", alpha=0.5, label="50th percentile")
    ax.axhline(75, color="#FF9800", ls="--", alpha=0.5, label="75th percentile")
    ax.set_title("Joshua's Percentile Ranking by Age Group", fontweight="bold")
    ax.set_ylabel("Percentile"); ax.set_ylim(0, 105); ax.legend()
    for bar, val in zip(bars, pct_df["Percentile"]):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2,
                f'{val:.0f}th', ha='center', fontweight='bold')
    plt.tight_layout(); plt.show()